In [61]:
import math
import requests
import datetime
from statistics import mean
import time
import sys

YEAR = 2020
WEEK = 11
POST_WEEK = 0
LI_RATIO = .1
L_SHARE = LI_RATIO
I_SHARE = 1 - LI_RATIO
TRANSFER_RATIO = 1
MARGIN_LOG = 7

In [62]:
#prev = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=regular')
#prev_post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR - 1) + '&seasonType=postseason')
reg = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=regular')
post = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=postseason')
records = {}
games = reg.json() + post.json() #+ prev.json() + prev_post.json()

In [63]:
for game in games:
        
    away_team = game['away_team']
    home_team = game['home_team']
    
    # With this dataset, no conference listed means they are an FCS team. 
    # I treat all FCS teams as the same, likely crappy team. 
    if not game['away_conference']:
        away_team = 'FCS Team'
    if not game['home_conference']:
        home_team = 'FCS Team'   
    
    if away_team not in records:
        records[away_team] = []
    if home_team not in records:
        records[home_team] = []
    
    if ((game['season_type'] != 'postseason'and game['week'] > WEEK) 
        or (game['season_type'] == 'postseason' and game['week'] > POST_WEEK)):
        continue
    
    # For invalid/unplayed games
    if game['away_points'] == None:
        continue
        
    margin = game['away_points'] - game['home_points']
    
    # Going to overtime counts as a one point win
    if len(game['away_line_scores']) > 4:
        margin = margin / abs(margin)
    records[away_team].append((home_team, margin))
    records[home_team].append((away_team, -margin))

In [105]:
AP_Ranks = {}
for team in records:
    AP_Ranks[team] = '-'
reg = requests.get('https://api.collegefootballdata.com/rankings?year=' + str(YEAR) 
                   + '&week=' + str(WEEK + POST_WEEK + 1)
                   + '&seasonType=regular')
for e in reg.json()[0]['polls']:
    if e['poll'] == 'AP Top 25':
        for row in e['ranks']:
            AP_Ranks[row['school']] = str(row['rank'])

In [65]:
records['Northwestern']

[('Maryland', 40), ('Iowa', 1), ('Nebraska', 8), ('Purdue', 7)]

In [66]:
# Get last weeks result!
last_week_table = {}
with open('Output - ' + str(YEAR) + '-' + str(WEEK + POST_WEEK - 1) + '.md', 'r') as last_week:
    rows = last_week.read().split('\n')
    header = []
    for row in rows:
        if '|' in row:
            cells = row.split('|')
            if header == []:
                header = cells
                team_loc = header.index('Team')
            else:
                team = '('.join(cells[team_loc].split('(')[0:-1]) # Get the team name without record, but with (OH)
                last_week_table[team] = {}
                for i, cell in enumerate(cells):
                    if cell == '':
                        continue
                    last_week_table[team][header[i]] = cell
                
                
                

In [67]:
last_week_table

{'': {'Rank': '-', 'Score': '-', 'Team': '-', 'Breakdown': '-'},
 'Coastal Carolina': {'Rank': '1',
  'Score': '1223',
  'Team': 'Coastal Carolina(7-0)',
  'Breakdown': '33 from #122 Kansas(15), 21 from #128 FCS Team(22), 204 from #58 Arkansas State(29), 385 from #6 Louisiana(3), 304 from #20 Georgia Southern(14), 213 from #60 Georgia State(51), 64 from #107 South Alabama(17)'},
 'Cincinnati': {'Rank': '2',
  'Score': '1214',
  'Team': 'Cincinnati(6-0)',
  'Breakdown': '23 from #128 FCS Team(35), 249 from #31 Army(14), 29 from #124 South Florida(21), 401 from #16 SMU(29), 336 from #25 Memphis(39), 176 from #64 Houston(28)'},
 'Alabama': {'Rank': '3',
  'Score': '1183',
  'Team': 'Alabama(6-0)',
  'Breakdown': '99 from #90 Missouri(19), 448 from #12 Texas A&M(28), 81 from #97 Ole Miss(15), 370 from #14 Georgia(17), 102 from #93 Tennessee(31), 83 from #104 Mississippi State(41)'},
 'BYU': {'Rank': '4',
  'Score': '1133',
  'Team': 'BYU(8-0)',
  'Breakdown': '90 from #102 Navy(52), 239 fr

In [94]:
start = time.time()
batch_start = time.time()

rankings = {}
scores = {}
for team in records:
    rankings[team] = 0
    scores[team] = [0]

team_count = len(rankings.keys())

rank_history = [] # used for checking if we have hit a repeating pattern based on ranks
past_averaged_history = '' # used to see if the patterns are broadly repeating
rank_history_set = set() 
score_history = [] # used for easier lookups of what team scores were in said repeating pattern
averaged_score_history = [] 
scores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)
repeat_point = 0
pattern_found = False
# Repeat for 100000 interations, but it should hit a repeating steady state before then
for i in range(1000001):
    for team in records.keys():
        scores[team] = [0] # the first value is the total score
        games = records[team]
        for game in games:
            opponent = game[0]
            margin = game[1]
            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1
            multiplier *= 1000 # just for nicer numbers
            game_score = 0
            if margin < 0:
                game_score -= L_SHARE * rankings[opponent] / team_count
                game_score -= I_SHARE / (team_count - rankings[opponent])
                if rankings[opponent] > rankings[team]:
                    game_score -= TRANSFER_RATIO * L_SHARE * (team_count - rankings[team]) / team_count
                    game_score -= TRANSFER_RATIO * I_SHARE / (1 + rankings[team])
            if margin > 0:
                game_score += L_SHARE * (team_count - rankings[opponent]) / team_count
                game_score += I_SHARE / (1 + rankings[opponent])
                if rankings[opponent] < rankings[team]:
                    game_score += TRANSFER_RATIO * L_SHARE * rankings[team] / team_count
                    game_score += TRANSFER_RATIO * I_SHARE / (team_count - rankings[team])
            scores[team] += [multiplier * game_score]
        if i == 0:
            scores[team][0] = sum(scores[team])
        elif i == 1:
            scores[team][0] = (sum(scores[team]) + score_history[-1][team][0]) / 2
        else:
            scores[team][0] = (sum(scores[team]) 
                               + (2 * score_history[-1][team][0] 
                                  - score_history[-2][team][0])) / 2
    # This is sorted, but adjusting so tied teams have the same (higher) rank
    new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
    #print(new_rank_orders)
    new_ranks = rankings
    new_ranks[new_rank_orders[0]] = 0
    past_rank = 0
    past_score = scores[new_rank_orders[0]][0]
    for rank, team in enumerate(new_rank_orders):
        if (abs(scores[team][0] - past_score) < 0.01):
            new_ranks[team] = past_rank
        else:
            new_ranks[team] = rank
        past_rank = new_ranks[team]
        past_score = scores[team][0]
    # stop if we are repeating ourselves, record this if we are not
    if False:
        repeat_point = rank_history.index(str(new_ranks))
        pattern_found = True
        break
    else:
        rank_history += [str(new_ranks)]
        rank_history_set.add(str(new_ranks))
        score_history += [scores.copy()]
        rankings = new_ranks
        
    # stop if we are *broadly* repeating ourselves
    if i%1000 == 0 and i != 0:
        print(i, 'Iterations Took: ' + str(round(time.time() - batch_start)))
        check_start = time.time()
        
        for team in records:
            score = [x[team] for x in score_history]
            scores[team] = list(map(mean, zip(*score)))
            scores[team][0] = sum(scores[team][1:]) 
        new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
        if str(new_rank_orders) == past_averaged_history:
            pattern_found = True
            break
        else:
            averaged_score_history += [scores.copy()]
            past_averaged_history = str(new_rank_orders)
        print(i, 'Check Took: ' + str(round(time.time() - check_start)))
        batch_start = time.time()
        
        
        
    if i>=1000:
        old_news = rank_history[0]
        del rank_history[0]
        del score_history[0]
            

print(i)



1000 Iterations Took: 2
1000 Check Took: 1
2000 Iterations Took: 2
2000 Check Took: 1
3000 Iterations Took: 2
3000 Check Took: 1
4000 Iterations Took: 2
4000 Check Took: 1
5000 Iterations Took: 2
5000 Check Took: 2
6000 Iterations Took: 2
6000 Check Took: 1
7000 Iterations Took: 2
7000 Check Took: 1
8000 Iterations Took: 2
8000 Check Took: 1
9000 Iterations Took: 2
9000 Check Took: 1
10000 Iterations Took: 2
10000 Check Took: 1
11000 Iterations Took: 2
11000 Check Took: 1
12000 Iterations Took: 2
12000 Check Took: 1
13000 Iterations Took: 2
13000 Check Took: 1
14000 Iterations Took: 2
14000 Check Took: 2
15000 Iterations Took: 2
15000 Check Took: 1
16000 Iterations Took: 2
16000 Check Took: 1
17000 Iterations Took: 2
17000 Check Took: 1
18000 Iterations Took: 2
18000 Check Took: 1
19000 Iterations Took: 2
19000 Check Took: 1
20000 Iterations Took: 2
20000 Check Took: 1
21000 Iterations Took: 2
21000 Check Took: 1
22000 Iterations Took: 2
22000 Check Took: 2
23000 Iterations Took: 2
230

In [110]:
# average across repeated cycled
for team in records:
    score = []
    if pattern_found: 
        for i in range(repeat_point, len(rank_history)):
            score += [score_history[i][team]]
    else:
        score = [x[team] for x in averaged_score_history]
    scores[team] = list(map(mean, zip(*score)))
    scores[team][0] = sum(scores[team][1:]) 
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ('My Computer Poll!\n'
          + 'It only takes into account games played this season, looking at margin and quality of wins and losses individually:\n\n'
          + '|Rank|Score|Team|AP Rank|Past Rank|Score Change|Breakdown|\n'
          + '|-|-|-|-|\n')
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(0, len(scores[team]) - 1):
        explanation += (str(round(scores[team][j + 1])) + ' from #'
                        + str(new_rank_orders.index(records[team][j][0]) + 1) + ' '
                        + str(records[team][j][0]) 
                        + '(' + str(records[team][j][1]) + '), ')
        
    score_change = round(scores[team][0]) - int(last_week_table[team]['Score'])
    if score_change > 0:
        score_change_text = '+' + str(score_change)
    else:
        score_change_text = str(score_change)
        
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + AP_Ranks[team] +'|'
            + last_week_table[team]['Rank'] + '|'
            + score_change_text + '|'
            + explanation[0:-2] + '\n')
    output += line
print(output)
with open('Output - '
        + str(YEAR) 
          + '-' 
          + str(WEEK) + '.md', 'w') as text_file:
    text_file.write(output)

My Computer Poll!
It only takes into account games played this season, looking at margin and quality of wins and losses individually:

|Rank|Score|Team|AP Rank|Past Rank|Score Change|Breakdown|
|-|-|-|-|
|1|1447|BYU(8-0)|8|4|+314|205 from #60 Navy(52), 223 from #54 Troy(41), 178 from #64 Louisiana Tech(31), 95 from #82 UT San Antonio(7), 277 from #26 Houston(17), 47 from #118 Texas State(38), 48 from #117 Western Kentucky(31), 375 from #19 Boise State(34)
|2|1353|Cincinnati(7-0)|7|2|+139|22 from #128 FCS Team(35), 159 from #63 Army(14), 37 from #120 South Florida(21), 421 from #14 SMU(29), 348 from #24 Memphis(39), 306 from #26 Houston(28), 60 from #113 East Carolina(38)
|3|1250|Coastal Carolina(7-0)|15|1|+27|29 from #123 Kansas(15), 20 from #128 FCS Team(22), 190 from #61 Arkansas State(29), 417 from #5 Louisiana(3), 296 from #21 Georgia Southern(14), 200 from #62 Georgia State(51), 98 from #90 South Alabama(17)
|4|1072|Alabama(6-0)|1|3|-111|99 from #87 Missouri(19), 393 from #16 Texa

In [95]:
ranks = []
for i in range (5):
    ranks.append(sorted(scores, key=averaged_score_history[-i].__getitem__, reverse=True))
for i in range(len(ranks[0])):
    row = str(i) + ' '
    for j in range (5):
        row += ranks[j][i] + ' '
    print(row)

0 BYU BYU BYU BYU BYU 
1 Cincinnati Cincinnati Cincinnati Cincinnati Cincinnati 
2 Coastal Carolina Coastal Carolina Coastal Carolina Coastal Carolina Coastal Carolina 
3 Alabama Alabama Alabama Alabama Alabama 
4 Clemson Louisiana Louisiana Louisiana Louisiana 
5 Notre Dame Clemson Clemson Clemson Clemson 
6 Louisiana Marshall Marshall Marshall Marshall 
7 Marshall Notre Dame Notre Dame Notre Dame Notre Dame 
8 Miami Miami Miami Miami Miami 
9 Oklahoma State Oklahoma State Oklahoma State Oklahoma State Oklahoma State 
10 Tulsa Tulsa Tulsa Tulsa Tulsa 
11 Northwestern Northwestern Northwestern Northwestern Northwestern 
12 San José State San José State San José State San José State San José State 
13 SMU SMU SMU SMU SMU 
14 Florida Florida Florida Florida Florida 
15 Texas A&M Texas A&M Texas A&M Texas A&M Texas A&M 
16 UCF UCF UCF UCF UCF 
17 Appalachian State Iowa State Iowa State Iowa State Iowa State 
18 Iowa State Boise State Boise State Boise State Boise State 
19 Boise State App